In [12]:
!pip install apache_beam

In [13]:
import pandas as pd
import apache_beam as beam

In [14]:
import subprocess
import os
import json
import gzip

In [15]:
from apache_beam.options.pipeline_options import PipelineOptions

beam_options = PipelineOptions()


Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')
Install apache-beam.
run('pip install --quiet apache-beam')

Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv')
```
# This is formatted as code
```



In [16]:
gsutil_path = 'gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv'
local_path = '/tmp/transactions.csv'

result = subprocess.run(['gsutil', 'cp', gsutil_path, local_path], check=True, capture_output=True, text=True)
print(result.stdout, result.stderr)

df = pd.read_csv(local_path)


 Copying gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv...
/ [0 files][    0.0 B/  570.0 B]                                                
/ [1 files][  570.0 B/  570.0 B]                                                
Operation completed over 1 objects/570.0 B.                                      



In [18]:
with beam.Pipeline() as pipeline:
  class ParseCSV(beam.DoFn):
    def process(self, element):
        fields = element.split(',')
        try:
            date = datetime.datetime.strptime(fields[0], '%Y-%m-%d').date()
            transaction_id = fields[1]
            product_id = fields[2]
            transaction_amount = float(fields[3])
            yield {
                'date': date,
                'transaction_id': transaction_id,
                'product_id': product_id,
                'transaction_amount': transaction_amount
            }
        except:
            pass
    class FilterTransactions(beam.DoFn):
      def process(self, element):
          if element['transaction_amount'] > 20 and element['date'].year >= 2010:
              yield element
    class SumTransactionsByDate(beam.DoFn):
      def process(self, element):
          date, transactions = element
          total_amount = sum(t['transaction_amount'] for t in transactions)
          yield {
                  'date': date.isoformat(),
                  'total_amount': total_amount
              }
def run():
    input_file = 'gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv'
    output_file = 'output/results.jsonl.gz'